In [1]:
import pandas as pd
import numpy as np
import requests
from get_mail_mailru import main as mail_main
from parser_broker_report import main as parser_main

In [84]:
# mail_main()
# parser_main()

In [2]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус', 'Номер сделки'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)
transactions_executed = transactions[transactions['Статус'] == 'И']

In [4]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A102HB1': 'SU26227RMFS7'
}

In [5]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A102HB1':
            new_ticker = 'SU26227RMFS7'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]


fix_split()

In [46]:
def get_stock_data_dict(tickers: list, transactions_executed):

    round_numb_3_list = ['LKOH', 'MGNT']

    round_numb_4_list = ['IRAO', 'MOEX', 'YDEX', 'SBMM']

    round_numb_5_list = ['HYDR', 'AFKS']

    round_numb_6_list = ['GAZP', 'MTSS', 'NVTK', 'ROSN', 'SBER', 'CHMF', 'SNGS', 'SBGD', 'SBMX', 'AFKS', 'AFLT', 'RTKM']

    # stock_data_dict = {}
    share_amount_dict = {}
    share_price_dict = {}
    commission_dict = {}

    for ticker in tickers:

        if ticker in round_numb_3_list:
            round_numb = 3
        elif ticker in round_numb_4_list:
            round_numb = 4
        elif ticker in round_numb_5_list:
            round_numb = 5
        elif ticker in round_numb_6_list:
            round_numb = 6
        else:
            round_numb = 2

        share_frame = transactions_executed[transactions_executed['Код'] == ticker]

        share_list = list(zip(share_frame['Вид'], share_frame['Количество'], share_frame['Сумма'], share_frame['Комиссия Брокера'], share_frame['Комиссия Биржи']))

        amount = 0
        price_avg = 0 
        total_cost = 0
        commission = 0

        for i in range(len(share_list)):

            type = share_list[i][0]
            amount_new = share_list[i][1]
            total_cost_new = share_list[i][2]
            commission_new = share_list[i][3] + share_list[i][4]

            if type == 'Покупка':

                amount += amount_new
                total_cost += total_cost_new
                commission += commission_new
                price_avg = round((total_cost + commission) / amount, round_numb)

            elif type == 'Продажа':

                amount -= amount_new
                total_cost = amount * price_avg

                if amount > 0:
                    commission += commission_new
                else:
                    commission = 0

            else:
                raise Exception('Неверный вид транзакции')

        share_amount_dict[ticker] = amount
        share_price_dict[ticker] = price_avg
        commission_dict[ticker] = round(commission, 2)

    return share_amount_dict, share_price_dict, commission_dict

share_amount_dict, share_price_dict, commission_dict = get_stock_data_dict(ticker_list, transactions_executed)

In [43]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict


def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A102HB1':
            ticker =  'SU26227RMFS7'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict


last_prices_dict = get_last_prices_dict(ticker_list)

In [44]:
def get_coupon_dict(tickers: list):
    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]
    return coupon_dict


coupon_dict = get_coupon_dict(ticker_list)

In [47]:
portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}

main = pd.DataFrame.from_dict(portfolio_dict)
main.index.name = 'Название'
main.loc[main['Котировки'] == '', 'Котировки'] = np.nan
main.dropna(axis=0, subset='Котировки', inplace=True)
main[['Котировки', 'НКД']] = main[['Котировки', 'НКД']].astype('float64')
# main.loc['SU25084RMFS3', 'Котировки'] = main.loc['SU25084RMFS3', 'Котировки'] * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (main.loc['SU25084RMFS3', 'Котировки'] + main.loc['SU25084RMFS3', 'НКД']) * main.loc['SU25084RMFS3', 'Количество']
main['Текущая цена'] = main['Котировки'] * main['Количество']
main['P/L, руб.'] = main['Текущая цена'] - main['Средняя цена'] * main['Количество']
main['P/L, %'] = (main['Котировки'] * 100 / main['Средняя цена'] - 100).round(2)

# bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
#             main.loc['SU25084RMFS3', 'Котировки'], \
#             main.loc['SU25084RMFS3', 'НКД'], \
#             main.loc['SU25084RMFS3', 'Количество'], \
#             main.loc['SU25084RMFS3', 'Средняя цена'], \
#             main.loc['SU25084RMFS3', 'Комиссия']

# main.loc['SU25084RMFS3', 'Котировки'] = bond_price * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (bond_price * 10 + bond_NKD) * bond_amount
# main.loc['SU25084RMFS3', 'P/L, руб.'] = main.loc['SU25084RMFS3', 'Текущая цена'] - bond_average_price * bond_amount
# main.loc['SU25084RMFS3', 'P/L, %'] = (((bond_price * 10 + bond_NKD) * 100 / bond_average_price) - 100).round(2)

main.to_csv('portfolio_main.csv', index_label=False)

In [48]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'

In [49]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,17.6430,NaN,1400,17.312579,7.04,24700.200,462.589400,1.91
SBER,275.5100,NaN,150,266.495667,33.45,41326.500,1352.149950,3.38
MTSS,209.0500,NaN,100,254.623000,21.30,20905.000,-4557.300000,-17.90
GAZP,125.1800,NaN,90,167.589667,13.57,11266.200,-3816.870030,-25.31
LKOH,7074.0000,NaN,4,6694.875000,17.50,28296.000,1516.500000,5.66
MGNT,4826.0000,NaN,5,5782.144000,23.72,24130.000,-4780.720000,-16.54
CHMF,1263.0000,NaN,14,1458.571429,17.00,17682.000,-2738.000006,-13.41
AFKS,14.8950,NaN,1200,17.475590,18.41,17874.000,-3096.708000,-14.77
IRAO,3.7155,NaN,4300,4.123200,14.74,15976.650,-1753.110000,-9.89
